In [1]:
import os
from bs4 import BeautifulSoup

# HW1:  Сравнение стилей текстов
### Выполнили:  Булгаков Дмитрий, Тефикова Алие
### Группа ИАД-2

# 1. Loading data from file 

Составьте самостоятельно как минимум две коллекции
текстов разных стилей (например, коллекция текстов в публицистическом
стиле и коллекция текстов в научном стиле). Коллекции текстов
должны быть достаточно большие (порядка 5000 токенов). Посчитайте
количество токенов и типов в каждой коллекции.

In [2]:
def remove_control_characters(text_string):
    return ''.join(filter(None, text_string.splitlines()))

### 1.1 Reading fiction text from file (War and Peace by Leo Tolstoy)

In [3]:
fiction = open('data/wap.txt', encoding='utf-8').read()
fiction = remove_control_characters(fiction)

### 1.2 Parsing journalistic style texts from Returers article corpus

In [4]:
tags_to_remove = ['date', 'topics', 'places', 'people', 'orgs', 'exchanges', 'companies', 
                  'unknown', 'title', 'dateline']
reuters_stopwords = ['reuters']

In [5]:
def parseXmlFile(path, tags, stopwords):
    xml_soup = BeautifulSoup(open(path), 'lxml')
    
    for trash in xml_soup(tags):
        trash.extract()
        
    parsed_text = xml_soup.get_text() # getting text from file 
    parsed_text = remove_control_characters(parsed_text) # removing control characters    
    parsed_text_words = [word for word in parsed_text.split() if word.lower() not in stopwords] # removing stop words
    return ' '.join(parsed_text_words)

In [6]:
jourrnalistic = parseXmlFile('data/reuters.txt', tags=tags_to_remove, stopwords=reuters_stopwords)